In [17]:
pip install -U kaleido

Note: you may need to restart the kernel to use updated packages.


In [18]:
import requests
import pandas as pd
import plotly.express as px


## Basic vaiables

In [19]:
eu_member_states = ["Austria", "Belgium", "Bulgaria", "Croatia", "Czechia", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "Netherlands", "Poland", "Portugal", "Romania", "Slovak Republic", "Slovenia", "Spain", "Sweden"]

eu_countries_abbr = ['AUT', 'BEL', 'BGR', 'HRV', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE']

## Investments In Roads by Country and Year


In [20]:
# Pull from API into JSON
def get_infr_data():
    """ Get the road spending data for all countries from 1995 to 2023

    Params: None

    Returns: JSON infr_response that has all of the data scraped directly from the API.
    """

    # Base URL
    base_url = "https://sdmx.oecd.org/public/rest/data"

    # Modifiers
    agency = "OECD.ITF"
    dataset = "DSD_INFRINV@DF_INFRINV"
    version = "1.0"  # Assuming version 1.0, could be different
    dimensions = ".A.INV.EUR.ROAD.V"

    # Parameters
    params = {
        "startPeriod": "1995",
        "endPeriod": "2023",
        "dimensionAtObservation": "AllDimensions",
        "format": "jsondata"
    }

    # Construct the URL
    url = f"{base_url}/{agency},{dataset},{version}/{dimensions}"

    # Make the request
    infr_response = requests.get(url, params=params).json()
    return infr_response


In [21]:
# Create dictionary of all the numerical data in the table
def create_infr_info_dict(infr_json):
    all_info_dict = infr_json["data"]["dataSets"][0]["observations"]

    # Create the dictionary in the shape of the table
    infr_dict = {}
    for i in range(29):
        infr_dict[i] = [None for j in range(55)]

    for key in all_info_dict.keys():
        key_list = key.split(":")
        infr_dict[int(key_list[6])][int(key_list[0])] = all_info_dict[key][0]

    return infr_dict

In [22]:
# Create the list of countries in order
def get_list_of_countries_in_order(countries_info_dict):
    countries_in_order = []

    for i in range(len(countries_info_dict)):
        countries_in_order.append(countries_info_dict[i]["name"])

    return countries_in_order

In [23]:
# Create the DataFrame, edit the column headers by year, add the country column.
def create_and_clean_infr_df(infr_dict, countries_in_order):
    # Create df, edit columns headers to years, put country names in
    infr_df = pd.DataFrame(infr_dict)
    infr_df = infr_df.set_axis([int(infr_df.keys()[i]) + 1995 for i in range(29)], axis=1)
    infr_df.insert(loc=0, column="Country", value=countries_in_order)

    # Set it to only EU Countries
    for i in range(len(infr_df) - 1, -1, -1):
        if infr_df.iloc[i].Country not in eu_member_states:
            infr_df = infr_df.drop([i], axis=0)

    # Sort and index by country
    infr_df.sort_values("Country", inplace=True)
    infr_df.set_index("Country", inplace=True)
    return infr_df

In [24]:
# Pull from API, organize data into a dict, and create and filter DataFrame
infr_data_json = get_infr_data()
infr_dict = create_infr_info_dict(infr_data_json)
countries_info_dict = infr_data_json["data"]["structures"][0]["dimensions"]["observation"][0]["values"]
countries_in_order = get_list_of_countries_in_order(countries_info_dict)
infr_df = create_and_clean_infr_df(infr_dict, countries_in_order)

infr_df

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,,,,,,,,,,,,,
Austria,4.570000e+08,4.260000e+08,3.650000e+08,4.300000e+08,3.910000e+08,4.750000e+08,6.400000e+08,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Belgium,1.600000e+08,1.590000e+08,1.530000e+08,1.520000e+08,1.610000e+08,1.490000e+08,1.470000e+08,1.720000e+08,1.850000e+08,1.560000e+08,...,4.170000e+08,7.780000e+08,8.100000e+08,6.559060e+08,6.810000e+08,7.350000e+08,6.070000e+08,1.311000e+09,NaN,NaN
Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.634523e+08,8.390428e+08,1.631046e+08,1.150424e+08,8.825033e+08,5.281726e+08,5.251048e+08,1.518560e+08,4.039268e+08,NaN
Croatia,6.423784e+07,1.236977e+08,1.709470e+08,3.227819e+08,3.420267e+08,3.087133e+08,3.026080e+08,6.057469e+08,1.052226e+09,8.722543e+08,...,2.832305e+08,2.408919e+08,1.973588e+08,1.947044e+08,2.811069e+08,3.493264e+08,4.422324e+08,5.365983e+08,4.709005e+08,6.204730e+08
Czechia,2.825142e+08,3.057651e+08,3.820312e+08,3.746393e+08,3.222859e+08,3.086187e+08,3.024600e+08,5.182849e+08,6.256379e+08,1.031226e+09,...,6.040137e+08,8.853583e+08,8.492317e+08,9.841913e+08,1.044764e+09,1.383065e+09,1.640906e+09,1.817147e+09,1.943388e+09,2.625930e+09
Denmark,3.518445e+08,4.036794e+08,3.995777e+08,3.877437e+08,4.194685e+08,4.545333e+08,4.970344e+08,3.991656e+08,5.866204e+08,7.278324e+08,...,1.101557e+09,1.086397e+09,1.099471e+09,1.065643e+09,1.083964e+09,NaN,NaN,NaN,NaN,NaN
Estonia,8.000000e+06,1.200000e+07,1.000000e+07,1.700000e+07,2.100000e+07,2.200000e+07,2.200000e+07,4.700000e+07,4.800000e+07,5.600000e+07,...,1.476600e+08,1.850500e+08,1.486000e+08,1.970000e+08,2.190000e+08,2.200000e+08,2.440000e+08,2.810000e+08,2.510000e+08,NaN
Finland,4.570000e+08,4.290000e+08,4.360000e+08,4.430000e+08,4.580000e+08,4.880000e+08,5.080000e+08,5.200000e+08,5.330000e+08,5.990000e+08,...,1.442000e+09,1.451000e+09,1.441000e+09,1.596000e+09,1.788000e+09,1.727000e+09,1.789000e+09,1.662000e+09,1.677000e+09,1.939000e+09
France,1.080527e+10,1.094766e+10,1.081952e+10,1.053208e+10,1.027529e+10,1.094036e+10,1.092041e+10,1.034518e+10,1.099093e+10,1.152636e+10,...,1.080720e+10,1.001123e+10,9.216000e+09,9.080000e+09,9.883630e+09,1.041300e+10,9.630572e+09,1.024821e+10,1.061250e+10,1.026531e+10


## GDP by Country

In [25]:
def bil_usd_to_euro(bil_usd):
    result = bil_usd * 1_000_000_000 # billions of usd to usd
    result *= 0.89 # usd to euro
    return result

In [26]:
def gdp_pull_API_into_dict():
    gdp_dict = {}
    for i in range(29):
        gdp_dict[i + 1995] = []

    for c in range(len(eu_countries_abbr)):
        # gdp_dict["Country"].append(eu_member_states[c])
        gdp_url = f'https://api.api-ninjas.com/v1/gdp?country={eu_countries_abbr[c]}'
        gdp_api_data = requests.get(gdp_url, headers={'X-Api-Key': '7DGbBN8IqcLbLHWaItioSA==jTkf7nL50oD4Ghnb'}).json()

        for i in range(len(gdp_api_data)):
            if 1995 <= gdp_api_data[i]["year"] <= 2023:

                gdp_in_euro = bil_usd_to_euro(gdp_api_data[i]["gdp_nominal"])

                gdp_dict[gdp_api_data[i]["year"]].append(gdp_in_euro)

    return gdp_dict

In [27]:
gdp_dict = gdp_pull_API_into_dict()

gdp_df = pd.DataFrame(gdp_dict)
# gdp_df.insert(loc=0, column="Country", value=eu_member_states)
gdp_df.index = eu_member_states
gdp_df

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Austria,2.146983e+11,2.112353e+11,1.896100e+11,1.945157e+11,1.935528e+11,1.756655e+11,1.757839e+11,1.906763e+11,2.333651e+11,2.681757e+11,...,3.940021e+11,3.399889e+11,3.521979e+11,3.712315e+11,4.051262e+11,3.957340e+11,3.868839e+11,4.268565e+11,4.195229e+11,4.607192e+11
Belgium,2.565879e+11,2.485699e+11,2.251896e+11,2.303996e+11,2.300650e+11,2.108383e+11,2.107057e+11,2.297980e+11,2.830236e+11,3.284518e+11,...,4.766208e+11,4.115209e+11,4.235786e+11,4.473024e+11,4.837559e+11,4.769732e+11,4.677840e+11,5.351614e+11,5.198250e+11,5.628351e+11
Bulgaria,1.690288e+10,1.094255e+10,1.007213e+10,1.337759e+10,1.212892e+10,1.178894e+10,1.262287e+10,1.459867e+10,1.881994e+10,2.328062e+10,...,5.080209e+10,4.518174e+10,4.799681e+10,5.276632e+10,5.909422e+10,6.131121e+10,6.257679e+10,7.484900e+10,8.047558e+10,9.043379e+10
Croatia,1.837850e+10,1.966633e+10,1.966366e+10,2.132707e+10,2.099065e+10,1.970282e+10,2.052607e+10,2.379771e+10,3.137161e+10,3.723404e+10,...,5.296034e+10,4.540424e+10,4.692614e+10,5.010878e+10,5.507854e+10,5.506519e+10,5.177931e+10,6.198939e+10,6.413251e+10,7.361279e+10
Czechia,5.390285e+10,6.034645e+10,5.565170e+10,5.977329e+10,5.837154e+10,5.533219e+10,6.064104e+10,7.351044e+10,8.941919e+10,1.069317e+11,...,1.877108e+11,1.683061e+11,1.763633e+11,1.971920e+11,2.242729e+11,2.285467e+11,2.234879e+11,2.589660e+11,2.686296e+11,3.054551e+11
Denmark,1.645165e+11,1.668590e+11,1.541854e+11,1.574196e+11,1.583194e+11,1.459983e+11,1.467441e+11,1.591213e+11,1.943956e+11,2.242693e+11,...,3.140214e+11,2.685655e+11,2.778420e+11,2.951338e+11,3.162108e+11,3.074078e+11,3.165116e+11,3.634564e+11,3.577319e+11,3.623119e+11
Estonia,3.479010e+09,4.258650e+09,4.592400e+09,5.053420e+09,5.136190e+09,5.079230e+09,5.583860e+09,6.579770e+09,8.805660e+09,1.082329e+10,...,2.408607e+10,2.074946e+10,2.185306e+10,2.443940e+10,2.780093e+10,2.837053e+10,2.829399e+10,3.312758e+10,3.417689e+10,3.675433e+10
Finland,1.194469e+11,1.175209e+11,1.130540e+11,1.194060e+11,1.204784e+11,1.121605e+11,1.152737e+11,1.248332e+11,1.526973e+11,1.756584e+11,...,2.434586e+11,2.075783e+11,2.124537e+11,2.258437e+11,2.438538e+11,2.376692e+11,2.401078e+11,2.620329e+11,2.496112e+11,2.631000e+11
France,1.420662e+12,1.423331e+12,1.291442e+12,1.334087e+12,1.324669e+12,1.211788e+12,1.219644e+12,1.327322e+12,1.632827e+12,1.876863e+12,...,2.547156e+12,2.174031e+12,2.198056e+12,2.303280e+12,2.476726e+12,2.423554e+12,2.354767e+12,2.641880e+12,2.490667e+12,2.716914e+12
Germany,2.309789e+12,2.231629e+12,1.977139e+12,2.003254e+12,1.972304e+12,1.751386e+12,1.750092e+12,1.869770e+12,2.255332e+12,2.537410e+12,...,3.530472e+12,3.047284e+12,3.147759e+12,3.347970e+12,3.607923e+12,3.522304e+12,3.503920e+12,3.872557e+12,3.708516e+12,4.029038e+12


In [28]:
def year_to_idx(year):
    return year - 1995

percent_dict = {}
for i in range(29):
    percent_dict[i + 1995] = []

for year in percent_dict.keys():
    for country in eu_member_states:
        percent_dict[year].append((infr_df.loc[country, year] / gdp_df.loc[country, year]) * 100)

infr_as_percent_of_gdp_df = pd.DataFrame(percent_dict)
infr_as_percent_of_gdp_df.insert(loc=0, column="Country", value=eu_member_states)
infr_as_percent_of_gdp_df.set_index("Country", inplace=True)
infr_as_percent_of_gdp_df

,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Country,,,,,,,,,,,,,,,,,,,,,
Austria,0.212857,0.201671,0.192500,0.221062,0.202012,0.270400,0.364083,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Belgium,0.062357,0.063966,0.067943,0.065972,0.069980,0.070670,0.069766,0.074848,0.065366,0.047496,...,0.087491,0.189055,0.191228,0.146636,0.140773,0.154097,0.129761,0.244973,NaN,NaN
Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.109112,1.857040,0.339824,0.218022,1.493383,0.861462,0.839137,0.202883,0.501925,NaN
Croatia,0.349527,0.628982,0.869355,1.513484,1.629424,1.566848,1.474262,2.545400,3.354072,2.342626,...,0.534797,0.530549,0.420573,0.388563,0.510375,0.634387,0.854072,0.865629,0.734262,0.842887
Czechia,0.524117,0.506683,0.686468,0.626767,0.552128,0.557756,0.498771,0.705049,0.699668,0.964378,...,0.321779,0.526040,0.481524,0.499103,0.465845,0.605156,0.734226,0.701693,0.723445,0.859678
Denmark,0.213866,0.241929,0.259154,0.246312,0.264951,0.311328,0.338708,0.250856,0.301766,0.324535,...,0.350791,0.404518,0.395718,0.361071,0.342798,NaN,NaN,NaN,NaN,NaN
Estonia,0.229950,0.281779,0.217751,0.336406,0.408863,0.433137,0.393993,0.714311,0.545104,0.517403,...,0.613051,0.891830,0.679996,0.806075,0.787743,0.775453,0.862374,0.848236,0.734414,NaN
Finland,0.382597,0.365041,0.385656,0.371003,0.380151,0.435091,0.440690,0.416556,0.349057,0.341003,...,0.592298,0.699013,0.678265,0.706683,0.733226,0.726640,0.745082,0.634271,0.671845,0.736982
France,0.760579,0.769158,0.837787,0.789460,0.775688,0.902828,0.895378,0.779402,0.673123,0.614129,...,0.424285,0.460492,0.419280,0.394220,0.399060,0.429658,0.408982,0.387914,0.426091,0.377830


In [29]:
final_gdp_roadspending_dict = {"Country": [], "Year": [], "RoadSpending": [], "GDP": [], "Spending/GDP%": []}

for country in eu_member_states:
    for year in percent_dict.keys():
        final_gdp_roadspending_dict["Country"].append(country)
        final_gdp_roadspending_dict["Year"].append(year)
        final_gdp_roadspending_dict["RoadSpending"].append(infr_df.loc[country, year])
        final_gdp_roadspending_dict["GDP"].append(gdp_df.loc[country, year])
        final_gdp_roadspending_dict["Spending/GDP%"].append(infr_as_percent_of_gdp_df.loc[country, year])

final_gdp_roadspending_df = pd.DataFrame(final_gdp_roadspending_dict)
final_gdp_roadspending_df

,Country,Year,RoadSpending,GDP,Spending/GDP%
0,Austria,1995,4.570000e+08,2.146983e+11,0.212857
1,Austria,1996,4.260000e+08,2.112353e+11,0.201671
2,Austria,1997,3.650000e+08,1.896100e+11,0.192500
3,Austria,1998,4.300000e+08,1.945157e+11,0.221062
4,Austria,1999,3.910000e+08,1.935528e+11,0.202012
...,...,...,...,...,...
749,Sweden,2019,2.503424e+09,4.736304e+11,0.528561
750,Sweden,2020,2.941620e+09,4.851817e+11,0.606292
751,Sweden,2021,2.910555e+09,5.670964e+11,0.513238
752,Sweden,2022,2.895628e+09,5.161074e+11,0.561051


In [30]:
gdp = final_gdp_roadspending_df['GDP']
road_spend = final_gdp_roadspending_df['RoadSpending']
country = final_gdp_roadspending_df["Country"]
year = final_gdp_roadspending_df['Year']
df = df = final_gdp_roadspending_df
fig = px.scatter(df , x= gdp, y= road_spend, animation_frame= year, size= gdp, color= country,
           hover_name= country, log_x=True, size_max=60, range_x=[5e+9,6e+12], range_y=[(-1000000000),2.5e+10])
fig.update_layout(
    xaxis_title="GDP",
    yaxis_title= "Road Spending", 
    title={
        "text": "Country GDP vs. Road Spending (1995-2023)",
        "x": 0.5,
        "xanchor": "center"
    }
)
fig.write_image("GDP_vs_Road_Spending_per_country.png")
fig.show()